In [1]:
import pandas as pd
import numpy as np

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
data_x=pd.read_csv("F:\Personal\Queens MMAI\MMAI 869 – Machine Learning and AI Technology\Driven Data team Assignment\Train set values.csv")
data_y=pd.read_csv("F:\Personal\Queens MMAI\MMAI 869 – Machine Learning and AI Technology\Driven Data team Assignment\Train set labels.csv")

In [4]:
data=pd.merge(data_x,data_y,on="id",how="inner")

## Splitting the data into training and testing

In [5]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(data_x,data_y,train_size=0.7,random_state=42,stratify=data_y["status_group"])

## Data Exploration

In [6]:
#Numerical Features Description
x_train.describe()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,41580.000000,41580.000000,41580.000000,41580.000000,4.158000e+04,41580.000000,41580.000000,41580.000000,41580.000000,41580.000000
mean,37053.095238,328.297768,670.883333,34.088923,-5.716324e+00,0.500000,15.198509,5.593218,180.270178,1303.064791
std,21379.253543,3380.560532,693.624472,6.541418,2.940352e+00,14.103926,17.463795,9.575067,462.724333,950.851187
min,0.000000,0.000000,-63.000000,0.000000,-1.164944e+01,0.000000,1.000000,0.000000,0.000000,0.000000
25%,18530.750000,0.000000,0.000000,33.097738,-8.543252e+00,0.000000,5.000000,2.000000,0.000000,0.000000
50%,37055.500000,0.000000,374.000000,34.911885,-5.036156e+00,0.000000,12.000000,3.000000,27.500000,1986.000000
75%,55351.250000,25.000000,1322.000000,37.169114,-3.328690e+00,0.000000,17.000000,5.000000,215.000000,2004.000000
max,74247.000000,350000.000000,2770.000000,40.344301,-2.000000e-08,1776.000000,99.000000,80.000000,15300.000000,2013.000000


In [7]:
#Categorical Features Description
x_train.describe(include="O")

,date_recorded,funder,installer,wpt_name,basin,subvillage,region,lga,ward,public_meeting,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
count,41580,39034,39025,41580,41580,41336,41580,41580,41580,39283,...,41580,41580,41580,41580,41580,41580,41580,41580,41580,41580
unique,346,1611,1802,27265,9,16078,21,124,2073,2,...,7,8,6,5,5,10,7,3,7,6
top,2011-03-17,Government Of Tanzania,DWE,none,Lake Victoria,Majengo,Iringa,Njombe,Igosi,True,...,never pay,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
freq,410,6405,12204,2546,7058,364,3752,1769,222,35748,...,17792,35496,35496,23232,23232,11944,11944,32085,19911,24183


In [8]:
x_train.ward.nunique()

2073

## Data Processing

In [9]:
# making a copy of x_train and y_train. All processing will be done on the copies.

In [10]:
x_train_3=x_train.copy()
y_train_3=y_train.copy()

In [11]:
def process2(a,b):
    ## new feature processing
    for i in ["funder","installer"]:
        a[i]=a[i].str.lower()
        minor_values=a[i].value_counts(dropna=False)[a[i].value_counts(dropna=False,normalize=True).cumsum().values>0.95].index
        a[i]=a[i].apply(lambda s:"minor" if s in minor_values else s)
        grp_mode=a.groupby(by=["region"])[i].transform(lambda x: x.mode().iloc[0])
        a[i]=a[i].fillna(grp_mode)
    a["long_mod"]=a["longitude"]
    a.loc[a['long_mod'] == 0, 'long_mod'] = pd.NA
    grp_means=a.groupby(by=["region"])["long_mod"].transform("mean")
    a.long_mod=a.long_mod.fillna(grp_means)
    a["lat_mod"]=a["latitude"]
    a.loc[a['lat_mod'] <-100, 'lat_mod'] = pd.NA
    grp_means2=a.groupby(by=["region"])["lat_mod"].transform("mean")
    a.lat_mod=a.lat_mod.fillna(grp_means2)
    a["gps_height_mod"]=a["gps_height"]
    a.loc[a['gps_height_mod'] == 0, 'gps_height_mod'] = pd.NA
    grp_means3=a.groupby(by=["basin"])["gps_height_mod"].transform("mean")
    a.gps_height_mod=a.gps_height_mod.fillna(grp_means3)
    a.wpt_name=a.wpt_name.str.lower()
    x=list(a.wpt_name.value_counts()[(a.wpt_name.value_counts().values==1)].index)
    a.wpt_name=a.wpt_name.replace(x,"single")
    #subvillage
    grp_mode4=a.groupby(by=["region"])["subvillage"].transform(lambda x: x.mode().iloc[0])
    a.subvillage=a.subvillage.fillna(grp_mode4)
    y=list(a.subvillage.value_counts()[(a.subvillage.value_counts().values==1)].index)
    a.subvillage=a.subvillage.replace(y,"single")
    a.public_meeting=a.public_meeting.fillna("True")
    grp_mode5=a.groupby(by=["region"])["permit"].transform(lambda x: x.mode().iloc[0])
    a.permit=a.permit.fillna(grp_mode5)
    grp_mode6=a.groupby(by=["region"])["scheme_management"].transform(lambda x: x.mode().iloc[0])
    a.scheme_management=a.scheme_management.fillna(grp_mode6)
    a["permit"]=a["permit"].map(lambda s: "T" if s==True else("F" if s== False else "M"))
    a["public_meeting"]=a["public_meeting"].map(lambda s: "T" if s==True else("F" if s== False else "M"))
    #bin_edges=[np.min(a.gps_height)-1,0,500,1000,1500,2000,np.max(a.gps_height)]
    #bin_labels=["<0","0-500","500-1000","1000-1500","1500-2000",">2000"]
    #a["gps_height_cat"]=pd.cut(a.gps_height,bins=bin_edges,labels=bin_labels).astype("O")
    a=a.drop(["id","recorded_by","scheme_name","num_private"],axis=1)
    b=b.drop("id",axis=1)
    a["amount_tsh_cat"] = a["amount_tsh"].apply(lambda x: 0 if x == 0 else 1)
    date_format = '%Y-%m-%d'
    a["date_recorded"]=pd.to_datetime(a["date_recorded"])
    a["dsr"]=pd.to_datetime("2023-11-16")-a["date_recorded"]
    a.dsr=(a.dsr/ pd.Timedelta(seconds=86400))
    bin_edges=[np.min(a.construction_year)-1,0,1961,1971,1981,1991,2001,np.max(a.construction_year)+1]
    bin_labels=["0","0-1960","1960-70","1970-80","1980-90","1990-2000","2001-2013"]
    a["construction_year_cat"]=pd.cut(a.construction_year,bins=bin_edges,labels=bin_labels).astype("O")
    a=a.drop(["amount_tsh","date_recorded","gps_height","latitude","longitude","construction_year"],axis=1)
    a[["region_code","district_code","amount_tsh_cat"]]=a[["region_code","district_code","amount_tsh_cat"]].astype("O")
    cat=a.select_dtypes(include="O").columns
    #a[cat] = a[cat].fillna('missing')
    a[["region_code","district_code","amount_tsh_cat"]]=a[["region_code","district_code","amount_tsh_cat"]].astype("O")
    return a,b

In [12]:
x_train_2,y_train_2=process2(x_train_3,y_train_3)

In [13]:
x_train.nunique()

id                       41580
amount_tsh                  93
date_recorded              346
funder                    1611
gps_height                2385
installer                 1802
longitude                40285
latitude                 40283
wpt_name                 27265
num_private                 58
basin                        9
subvillage               16078
region                      21
region_code                 27
district_code               20
lga                        124
ward                      2073
population                 948
public_meeting               2
recorded_by                  1
scheme_management           12
scheme_name               2472
permit                       2
construction_year           55
extraction_type             18
extraction_type_group       13
extraction_type_class        7
management                  12
management_group             5
payment                      7
payment_type                 7
water_quality                8
quality_

In [14]:
x_train_2.isna().sum()

funder                   0
installer                0
wpt_name                 0
basin                    0
subvillage               0
region                   0
region_code              0
district_code            0
lga                      0
ward                     0
population               0
public_meeting           0
scheme_management        0
permit                   0
extraction_type          0
extraction_type_group    0
extraction_type_class    0
management               0
management_group         0
payment                  0
payment_type             0
water_quality            0
quality_group            0
quantity                 0
quantity_group           0
source                   0
source_type              0
source_class             0
waterpoint_type          0
waterpoint_type_group    0
long_mod                 0
lat_mod                  0
gps_height_mod           0
amount_tsh_cat           0
dsr                      0
construction_year_cat    0
dtype: int64

In [15]:
x_train_2.columns

Index(['funder', 'installer', 'wpt_name', 'basin', 'subvillage', 'region',
       'region_code', 'district_code', 'lga', 'ward', 'population',
       'public_meeting', 'scheme_management', 'permit', 'extraction_type',
       'extraction_type_group', 'extraction_type_class', 'management',
       'management_group', 'payment', 'payment_type', 'water_quality',
       'quality_group', 'quantity', 'quantity_group', 'source', 'source_type',
       'source_class', 'waterpoint_type', 'waterpoint_type_group', 'long_mod',
       'lat_mod', 'gps_height_mod', 'amount_tsh_cat', 'dsr',
       'construction_year_cat'],
      dtype='object')

In [16]:
x_test_2=x_test.copy()
y_test_2=y_test.copy()
x_test_2,y_test_2=process2(x_test_2,y_test_2)

In [17]:
## Categorical features encoding
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
Label_col=x_train_2.select_dtypes(include="O").columns
Or=OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
Or.fit(x_train_2[Label_col])
x_train_2[Label_col]=Or.transform(x_train_2[Label_col])
x_test_2[Label_col]=Or.transform(x_test_2[Label_col])

In [18]:
le=LabelEncoder()
le.fit(y_train_2["status_group"])
y_train_2["status_group"]=le.transform(y_train_2["status_group"])
y_test_2["status_group"]=le.transform(y_test_2["status_group"])

In [19]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
def matrix(xtr,ytr,xte,yte,est):
    print ("\n Train accuracy",round(accuracy_score(ytr, est.predict(xtr)),3))
    print ("\n Train Classification Report\n\n", classification_report(ytr, est.predict(xtr)))
    print ("\n Test accuracy", round(accuracy_score(yte, est.predict(xte)),3))
    print ("\n Test Classification Report\n\n", classification_report(yte, est.predict(xte)))

In [20]:
from sklearn.ensemble import RandomForestClassifier

In [21]:
#cat_col=list(x_train_2.select_dtypes(include="O").columns)
#clf = CatBoostClassifier(iterations=10, depth=5, learning_rate=0.1, loss_function='MultiClass',cat_features=cat_col)

In [22]:
from sklearn.model_selection import cross_val_score

In [23]:
from sklearn.model_selection import GridSearchCV

In [24]:
y_train_2

,status_group
5974,0
31787,0
27800,0
8008,2
39725,2
...,...
21108,1
14847,2
26579,2
8192,0


In [25]:
#cat_col=list(x_train_2.select_dtypes(include="O").columns)
#clf = CatBoostClassifier(loss_function='MultiClass',cat_features=cat_col)
#parameters={"iterations":[1000],"depth":np.arange(2,20,2),"learning_rate":np.arange(0.01,0.1,0.01),"max_bin":np.arange(10,100,5)}
#Grid=GridSearchCV(clf,param_grid=parameters,cv=5,scoring="f1_macro",n_jobs=4)

In [26]:
%pip install optuna
import optuna

Note: you may need to restart the kernel to use updated packages.


In [88]:
def objective_dt(trial, X, y):

  # Now, define all the hyperparams we want to vary, and what values they are allowed
  # to take.
  #
  # Each trial, optuna will automatically choose values for each hyperparam.
  hyper_params = {
        "n_estimators":1000,
       #"max_depth":trial.suggest_int("max_depth",8,16,step=2),
        #"learning_rate":trial.suggest_float("learning_rate",0.01,.19,step=.01),
        "max_samples":trial.suggest_float("max_samples",0.4,0.8,step=0.1),
        #"max_bin":trial.suggest_int("max_bin",40,200,step=10),
        "min_samples_split":trial.suggest_int("min_samples_split",2,102,step=4),
        "random_state": 60,
  }


  # Use the hyperparams that optuna has chosen for this trial to create a DecisionTreeClassifier
  clf_rf = RandomForestClassifier(**hyper_params)

  # Run CV to see how well these hyper_params do
  cv_scores = cross_val_score(clf_rf, X, y, cv=5, scoring="f1_macro")
  score = np.mean(cv_scores)

  # Whatever we return here tells optuna how well these parameters did
  return score

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(lambda trial: objective_dt(trial, x_train_2,y_train_2.values.ravel()), n_trials=100,  gc_after_trial=True)

[I 2023-12-12 23:23:19,006] A new study created in memory with name: no-name-f5819eba-7ace-4c12-93c6-2428f56e5a7a
[I 2023-12-12 23:29:00,330] Trial 0 finished with value: 0.6259313274533415 and parameters: {'max_samples': 0.8, 'min_samples_split': 54}. Best is trial 0 with value: 0.6259313274533415.
[I 2023-12-12 23:32:58,932] Trial 1 finished with value: 0.6081272351338114 and parameters: {'max_samples': 0.5, 'min_samples_split': 58}. Best is trial 0 with value: 0.6259313274533415.
[I 2023-12-12 23:37:15,714] Trial 2 finished with value: 0.5970608824075018 and parameters: {'max_samples': 0.6000000000000001, 'min_samples_split': 86}. Best is trial 0 with value: 0.6259313274533415.
[I 2023-12-12 23:42:13,242] Trial 3 finished with value: 0.6168105841174321 and parameters: {'max_samples': 0.7000000000000001, 'min_samples_split': 62}. Best is trial 0 with value: 0.6259313274533415.
[I 2023-12-12 23:46:33,986] Trial 4 finished with value: 0.6375989137066937 and parameters: {'max_samples': 

In [ ]:
study.best_params

In [ ]:
{'n_estimators': 500,
 'criterion': 'log_loss',
 'max_depth': 22,
 'max_samples': 0.8,
 'min_samples_split': 2}

In [27]:
RF=RandomForestClassifier(n_estimators=500,max_samples=0.8,min_samples_split=2,criterion="log_loss",max_depth=22)

In [28]:
RF.fit(x_train_2,y_train_2.values.ravel())

RandomForestClassifier(criterion='log_loss', max_depth=22, max_samples=0.8,
                       n_estimators=500)

In [29]:
cv_scores = cross_val_score(RF, x_train_2, y_train_2.values.ravel(), cv=5, scoring="accuracy")

In [30]:
cv_scores

array([0.8044733 , 0.8047138 , 0.80555556, 0.80675806, 0.80772006])

In [31]:
cv_scores.mean()

0.8058441558441558

In [32]:
matrix(x_train_2,y_train_2,x_test_2,y_test_2,RF)


 Train accuracy 0.975

 Train Classification Report

               precision    recall  f1-score   support

           0       0.96      1.00      0.98     22581
           1       0.99      0.88      0.93      3022
           2       0.99      0.96      0.98     15977

    accuracy                           0.97     41580
   macro avg       0.98      0.95      0.96     41580
weighted avg       0.98      0.97      0.97     41580


 Test accuracy 0.814

 Test Classification Report

               precision    recall  f1-score   support

           0       0.80      0.91      0.85      9678
           1       0.62      0.31      0.41      1295
           2       0.86      0.77      0.81      6847

    accuracy                           0.81     17820
   macro avg       0.76      0.66      0.69     17820
weighted avg       0.81      0.81      0.81     17820



In [33]:
Target_x=pd.read_csv("F:\Personal\Queens MMAI\MMAI 869 – Machine Learning and AI Technology\Driven Data team Assignment\Test set values.csv")

In [34]:
Target_x

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,50785,0.0,2013-02-04,Dmdd,1996,DMDD,35.290799,-4.059696,Dinamu Secondary School,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,other,other
1,51630,0.0,2013-02-04,Government Of Tanzania,1569,DWE,36.656709,-3.309214,Kimnyak,0,...,never pay,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe
2,17168,0.0,2013-02-01,NaN,1567,NaN,34.767863,-5.004344,Puma Secondary,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,other,other
3,45559,0.0,2013-01-22,Finn Water,267,FINN WATER,38.058046,-9.418672,Kwa Mzee Pange,0,...,unknown,soft,good,dry,dry,shallow well,shallow well,groundwater,other,other
4,49871,500.0,2013-03-27,Bruder,1260,BRUDER,35.006123,-10.950412,Kwa Mzee Turuka,0,...,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14845,39307,0.0,2011-02-24,Danida,34,Da,38.852669,-6.582841,Kwambwezi,0,...,never pay,soft,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe
14846,18990,1000.0,2011-03-21,Hiap,0,HIAP,37.451633,-5.350428,Bonde La Mkondoa,0,...,annually,salty,salty,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump
14847,28749,0.0,2013-03-04,NaN,1476,NaN,34.739804,-4.585587,Bwawani,0,...,never pay,soft,good,insufficient,insufficient,dam,dam,surface,communal standpipe,communal standpipe
14848,33492,0.0,2013-02-18,Germany,998,DWE,35.432732,-10.584159,Kwa John,0,...,never pay,soft,good,insufficient,insufficient,river,river/lake,surface,communal standpipe,communal standpipe


In [35]:
Target1=Target_x.copy()

In [36]:
def process_target2(a):
    ## new feature processing
    for i in ["funder","installer"]:
        a[i]=a[i].str.lower()
        minor_values=a[i].value_counts(dropna=False)[a[i].value_counts(dropna=False,normalize=True).cumsum().values>0.95].index
        a[i]=a[i].apply(lambda s:"minor" if s in minor_values else s)
        grp_mode=a.groupby(by=["region"])[i].transform(lambda x: x.mode().iloc[0])
        a[i]=a[i].fillna(grp_mode)
    a["long_mod"]=a["longitude"]
    a.loc[a['long_mod'] == 0, 'long_mod'] = pd.NA
    grp_means=a.groupby(by=["region"])["long_mod"].transform("mean")
    a.long_mod=a.long_mod.fillna(grp_means)
    a["lat_mod"]=a["latitude"]
    a.loc[a['lat_mod'] <-100, 'lat_mod'] = pd.NA
    grp_means2=a.groupby(by=["region"])["lat_mod"].transform("mean")
    a.lat_mod=a.lat_mod.fillna(grp_means2)
    a["gps_height_mod"]=a["gps_height"]
    a.loc[a['gps_height_mod'] == 0, 'gps_height_mod'] = pd.NA
    grp_means3=a.groupby(by=["basin"])["gps_height_mod"].transform("mean")
    a.gps_height_mod=a.gps_height_mod.fillna(grp_means3)
    a.wpt_name=a.wpt_name.str.lower()
    x=list(a.wpt_name.value_counts()[(a.wpt_name.value_counts().values==1)].index)
    a.wpt_name=a.wpt_name.replace(x,"single")
    #subvillage
    grp_mode4=a.groupby(by=["region"])["subvillage"].transform(lambda x: x.mode().iloc[0])
    a.subvillage=a.subvillage.fillna(grp_mode4)
    y=list(a.subvillage.value_counts()[(a.subvillage.value_counts().values==1)].index)
    a.subvillage=a.subvillage.replace(y,"single")
    a.public_meeting=a.public_meeting.fillna("True")
    grp_mode5=a.groupby(by=["region"])["permit"].transform(lambda x: x.mode().iloc[0])
    a.permit=a.permit.fillna(grp_mode5)
    grp_mode6=a.groupby(by=["region"])["scheme_management"].transform(lambda x: x.mode().iloc[0])
    a.scheme_management=a.scheme_management.fillna(grp_mode6)
    a["permit"]=a["permit"].map(lambda s: "T" if s==True else("F" if s== False else "M"))
    a["public_meeting"]=a["public_meeting"].map(lambda s: "T" if s==True else("F" if s== False else "M"))
    #bin_edges=[np.min(a.gps_height)-1,0,500,1000,1500,2000,np.max(a.gps_height)]
    #bin_labels=["<0","0-500","500-1000","1000-1500","1500-2000",">2000"]
    #a["gps_height_cat"]=pd.cut(a.gps_height,bins=bin_edges,labels=bin_labels).astype("O")
    a=a.drop(["id","recorded_by","scheme_name","num_private"],axis=1)
    a["amount_tsh_cat"] = a["amount_tsh"].apply(lambda x: 0 if x == 0 else 1)
    date_format = '%Y-%m-%d'
    a["date_recorded"]=pd.to_datetime(a["date_recorded"])
    a["dsr"]=pd.to_datetime("2023-11-16")-a["date_recorded"]
    a.dsr=(a.dsr/ pd.Timedelta(seconds=86400))
    bin_edges=[np.min(a.construction_year)-1,0,1961,1971,1981,1991,2001,np.max(a.construction_year)+1]
    bin_labels=["0","0-1960","1960-70","1970-80","1980-90","1990-2000","2001-2013"]
    a["construction_year_cat"]=pd.cut(a.construction_year,bins=bin_edges,labels=bin_labels).astype("O")
    a=a.drop(["amount_tsh","date_recorded","gps_height","latitude","longitude","construction_year"],axis=1)
    a[["region_code","district_code","amount_tsh_cat"]]=a[["region_code","district_code","amount_tsh_cat"]].astype("O")
    cat=a.select_dtypes(include="O").columns
    #a[cat] = a[cat].fillna('missing')
    a[["region_code","district_code","amount_tsh_cat"]]=a[["region_code","district_code","amount_tsh_cat"]].astype("O")
    return a

In [37]:
Target1=process_target2(Target1)

In [38]:
Target1

,funder,installer,wpt_name,basin,subvillage,region,region_code,district_code,lga,ward,...,source_type,source_class,waterpoint_type,waterpoint_type_group,long_mod,lat_mod,gps_height_mod,amount_tsh_cat,dsr,construction_year_cat
0,dmdd,dmdd,single,Internal,Magoma,Manyara,21,3,Mbulu,Bashay,...,rainwater harvesting,surface,other,other,35.290799,-4.059696,1996.00000,0,3937.0,2001-2013
1,government of tanzania,dwe,single,Pangani,single,Arusha,2,2,Arusha Rural,Kimnyaki,...,spring,groundwater,communal standpipe,communal standpipe,36.656709,-3.309214,1569.00000,0,3937.0,1990-2000
2,tcrs,sema,single,Internal,single,Singida,13,2,Singida Rural,Puma,...,rainwater harvesting,surface,other,other,34.767863,-5.004344,1567.00000,0,3940.0,2001-2013
3,finn water,finn water,single,Ruvuma / Southern Coast,Kipindimbi,Lindi,80,43,Liwale,Mkutano,...,shallow well,groundwater,other,other,38.058046,-9.418672,267.00000,0,3950.0,1980-90
4,bruder,bruder,single,Ruvuma / Southern Coast,Losonga,Ruvuma,10,3,Mbinga,Mbinga Urban,...,spring,groundwater,communal standpipe,communal standpipe,35.006123,-10.950412,1260.00000,1,3886.0,1990-2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14845,danida,da,single,Wami / Ruvu,Yombo,Pwani,6,1,Bagamoyo,Yombo,...,river/lake,surface,communal standpipe,communal standpipe,38.852669,-6.582841,34.00000,0,4648.0,1980-90
14846,hiap,hiap,single,Pangani,Mkondoa,Tanga,4,7,Kilindi,Mvungwe,...,shallow well,groundwater,hand pump,hand pump,37.451633,-5.350428,1082.02754,1,4623.0,1990-2000
14847,tcrs,sema,bwawani,Internal,Juhudi,Singida,13,2,Singida Rural,Ughandi,...,dam,surface,communal standpipe,communal standpipe,34.739804,-4.585587,1476.00000,0,3909.0,2001-2013
14848,germany,dwe,kwa john,Lake Nyasa,Namakinga B,Ruvuma,10,2,Songea Rural,Maposeni,...,river/lake,surface,communal standpipe,communal standpipe,35.432732,-10.584159,998.00000,0,3923.0,2001-2013


In [39]:
Target1[Label_col]=Or.transform(Target1[Label_col])

In [40]:
y_pred=RF.predict(Target1)
y_pred=le.inverse_transform(y_pred)

In [41]:
y_pred=pd.DataFrame(y_pred, columns=["status_group"])
y_pred["id"]=Target_x["id"]
y_pred=y_pred.reindex(["id","status_group"],axis=1)
y_pred.to_csv("F:/Personal/Queens MMAI/MMAI 869 – Machine Learning and AI Technology/Driven Data team Assignment/RF 18.12.2023v2.csv",index=False)